If any `"path doesn't exist"` issue: Move this file to the same directory where `data` folder is placed.

In [1]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("ReadCSV").getOrCreate()

24/08/11 10:28:51 WARN Utils: Your hostname, szymon-Lenovo-Legion-5-15ARH05H resolves to a loopback address: 127.0.1.1; using 192.168.0.38 instead (on interface wlp4s0)
24/08/11 10:28:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/08/11 10:28:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Olympic Athlete Bio

In [2]:
# Read the CSV file into a DataFrame
Olympic_Athlete_Bio = spark.read.csv("data/Olympic_Athlete_Bio.csv", header=True, inferSchema=True)

In [3]:
# Show the DataFrame
Olympic_Athlete_Bio.show(1, vertical=True)

-RECORD 0-----------------------------
 athlete_id    | 65649                
 name          | Ivanka Bonova        
 sex           | Female               
 born          | 4 April 1949         
 height        | 166                  
 weight        | 55                   
 country       |  Bulgaria            
 country_noc   | BUL                  
 description   | Personal Best: 40... 
 special_notes | null                 
only showing top 1 row



In [4]:
Olympic_Athlete_Bio.printSchema()

root
 |-- athlete_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- born: string (nullable = true)
 |-- height: integer (nullable = true)
 |-- weight: string (nullable = true)
 |-- country: string (nullable = true)
 |-- country_noc: string (nullable = true)
 |-- description: string (nullable = true)
 |-- special_notes: string (nullable = true)



In [5]:
Olympic_Athlete_Bio.columns

['athlete_id',
 'name',
 'sex',
 'born',
 'height',
 'weight',
 'country',
 'country_noc',
 'description',
 'special_notes']

In [6]:
Olympic_Athlete_Bio.toPandas()

,athlete_id,name,sex,born,height,weight,country,country_noc,description,special_notes
0,65649,Ivanka Bonova,Female,4 April 1949,166.0,55,Bulgaria,BUL,Personal Best: 400 – 53.54 (1980).,None
1,112510,Nataliya Uryadova,Female,15 March 1977,184.0,70,Russian Federation,RUS,None,Listed in Olympians Who Won a Medal at the Eur...
2,114973,Essa Ismail Rashed,Male,14 December 1986,165.0,55,Qatar,QAT,Personal Best: 10000 – 27:20.97 (2006).,Listed in Olympians Who Won a Medal at the Asi...
3,30359,Péter Boros,Male,12 January 1908,NaN,None,Hungary,HUN,"Between 1927 and 1938, Péter Boros competed as...",None
4,50557,Rudolf Piowatý,Male,28 April 1900,NaN,None,Czechoslovakia,TCH,Rudolf Piowaty joined the Czechoslovak militar...,None
...,...,...,...,...,...,...,...,...,...,...
155856,23748,Todd Makler,Male,8 January 1946,183.0,75,United States,USA,"Todd Makler grew up in suburban Philadelphia, ...","Brother of Brooke Makler Son of Paul Makler, Sr."
155857,58581,Géza Hollósi,Male,2 May 1938,175.0,79,Hungary,HUN,Géza Hollósi had the following finishes at maj...,None
155858,30387,József Keresztessy,Male,19 September 1885,NaN,None,Hungary,HUN,József Keresztessy was the grandson of József ...,None
155859,69900,Alexander Thieme,Male,13 January 1954,187.0,84,East Germany,GDR,East German Sprinter Alexander Thieme reached ...,Listed in Olympians Who Won a Medal at the Eur...


In [7]:
from pyspark.sql.functions import upper

Olympic_Athlete_Bio.withColumn('COUNTRY_UPPER', upper(Olympic_Athlete_Bio.country)).show()

+----------+--------------------+------+----------------+------+------+-------------------+-----------+--------------------+--------------------+-------------------+
|athlete_id|                name|   sex|            born|height|weight|            country|country_noc|         description|       special_notes|      COUNTRY_UPPER|
+----------+--------------------+------+----------------+------+------+-------------------+-----------+--------------------+--------------------+-------------------+
|     65649|       Ivanka Bonova|Female|    4 April 1949|   166|    55|           Bulgaria|        BUL|Personal Best: 40...|                null|           BULGARIA|
|    112510|   Nataliya Uryadova|Female|   15 March 1977|   184|    70| Russian Federation|        RUS|                null|Listed in Olympia...| RUSSIAN FEDERATION|
|    114973|  Essa Ismail Rashed|  Male|14 December 1986|   165|    55|              Qatar|        QAT|Personal Best: 10...|Listed in Olympia...|              QATAR|
|   

In [8]:
print(f"Male athletes: {Olympic_Athlete_Bio.filter(Olympic_Athlete_Bio.sex == 'Male').count()}")
print(f"Female athletes: {Olympic_Athlete_Bio.filter(Olympic_Athlete_Bio.sex == 'Female').count()}")

Male athletes: 115527
Female athletes: 40334


In [104]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import round


print("Average athlete")
Olympic_Athlete_Bio_avg_height_by_sex = Olympic_Athlete_Bio.groupBy('sex').avg('height')
Olympic_Athlete_Bio_avg_height_by_sex.select('sex', round('avg(height)', 2).alias('avg_height')).toPandas()

Average athlete


,sex,avg_height
0,Female,168.91
1,Male,179.52


In [10]:
print('Total number of players 200cm tall or higher:')
Olympic_Athlete_Bio.filter(Olympic_Athlete_Bio.height >= 200).count()

Total number of players 200cm tall or higher:


1566

In [11]:
#TODO

from pyspark.sql.functions import min, max, length, row_number

# print(f'Player with longest name:')
# Olympic_Athlete_Bio.select(max(length('name')).alias('name_length'), Olympic_Athlete_Bio.name).show()

from pyspark.sql.window import Window

# Calculate name length and rank by length in descending order
Olympic_Athlete_Bio = Olympic_Athlete_Bio.withColumn('name_length', length('name'))
windowSpec = Window.orderBy(length('name').desc())
Olympic_Athlete_Bio = Olympic_Athlete_Bio.withColumn('rank', row_number().over(windowSpec))

# Get the longest name
longest_name_Olympic_Athlete_Bio = Olympic_Athlete_Bio.filter('rank = 1').select('name', 'name_length')

print(f'Player with longest name:')
longest_name_Olympic_Athlete_Bio.show()


print(f'Player with shortest name:')
Olympic_Athlete_Bio.select(min(Olympic_Athlete_Bio.name)).show()

Player with longest name:
24/08/11 10:29:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/11 10:29:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/11 10:29:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/11 10:29:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/11 10:29:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+--------------------+-----------+
|                name|name_length|
+--------------------+-----------+
|Georges, Marquis ...|         49|
+--------

In [12]:
from pyspark.sql import functions as F


Olympic_Athlete_Bio1 = Olympic_Athlete_Bio.withColumn('surname', F.split('name', ' ').getItem(1)) \
       .withColumn('surname_length', F.length('surname')) \
       .sort(F.desc('surname_length'))

Olympic_Athlete_Bio2 = Olympic_Athlete_Bio.withColumn('surname', F.split('name', ' ').getItem(1)) \
       .withColumn('surname_length', F.length('surname')) \
       .sort(F.asc('surname_length'))

print('longest surname')
Olympic_Athlete_Bio1.select('name', 'surname').show(1)

print('shortest surname') #TODO you'll have to clean the data (delete double spaces)
Olympic_Athlete_Bio2.select('name', 'surname').show(1)

longest surname
+--------------------+--------------------+
|                name|             surname|
+--------------------+--------------------+
|Martina Jašicová-...|Jašicová-Schwarzb...|
+--------------------+--------------------+
only showing top 1 row

shortest surname
+-------+-------+
|   name|surname|
+-------+-------+
|Cherry |       |
+-------+-------+
only showing top 1 row



In [13]:
print('heaviest and lightest male/female')
Olympic_Athlete_Bio.groupBy(Olympic_Athlete_Bio.sex).agg({'weight': 'max'}).show()

heaviest and lightest male/female
+------+-----------+
|   sex|max(weight)|
+------+-----------+
|Female|         99|
|  Male|     99-101|
+------+-----------+



In [15]:
print('Countries with most athletes:')
Olympic_Athlete_Bio.groupBy('country').count().sort('count', ascending=False).show(3)

Countries with most athletes:
+--------------+-----+
|       country|count|
+--------------+-----+
| United States|11691|
|        France| 7690|
| Great Britain| 6995|
+--------------+-----+
only showing top 3 rows



In [17]:
print('total number of athletes:')
Olympic_Athlete_Bio.count()

total number of athletes:


155861

In [18]:
print('Years when most athletes were born')
df = Olympic_Athlete_Bio.withColumn('Year', F.substring_index('born', ' ', -1))
df.filter(df.Year.isNotNull()).groupBy('Year').count().sort('count', ascending=False).show(3)

Years when most athletes were born
+----+-----+
|Year|count|
+----+-----+
|1972| 2289|
|1985| 2284|
|1973| 2274|
+----+-----+
only showing top 3 rows



### Olympic Games Medal Tally

In [19]:
# Read the CSV file into a DataFrame
Olympic_Games_Medal_Tally = spark.read.csv("data/Olympic_Games_Medal_Tally.csv", header=True, inferSchema=True)

In [20]:
Olympic_Games_Medal_Tally.show()

+--------------------+----------+----+-------------+-----------+----+------+------+-----+
|             edition|edition_id|year|      country|country_noc|gold|silver|bronze|total|
+--------------------+----------+----+-------------+-----------+----+------+------+-----+
|1896 Summer Olympics|         1|1896|United States|        USA|  11|     7|     2|   20|
|1896 Summer Olympics|         1|1896|       Greece|        GRE|  10|    18|    19|   47|
|1896 Summer Olympics|         1|1896|      Germany|        GER|   6|     5|     2|   13|
|1896 Summer Olympics|         1|1896|       France|        FRA|   5|     4|     2|   11|
|1896 Summer Olympics|         1|1896|Great Britain|        GBR|   2|     3|     2|    7|
|1896 Summer Olympics|         1|1896|      Hungary|        HUN|   2|     1|     3|    6|
|1896 Summer Olympics|         1|1896|      Austria|        AUT|   2|     1|     2|    5|
|1896 Summer Olympics|         1|1896|    Australia|        AUS|   2|     0|     0|    2|
|1896 Summ

In [21]:
print('Top 3 most total medals in a single Olympic Event:')
Olympic_Games_Medal_Tally.select('edition', 'country', 'total') \
    .sort('total', ascending=False) \
    .show(3)


Top 3 most total medals in a single Olympic Event:
+--------------------+-------------+-----+
|             edition|      country|total|
+--------------------+-------------+-----+
|1904 Summer Olympics|United States|  248|
|1980 Summer Olympics| Soviet Union|  195|
|1984 Summer Olympics|United States|  174|
+--------------------+-------------+-----+
only showing top 3 rows



In [22]:
Olympic_Games_Medal_Tally.printSchema()

root
 |-- edition: string (nullable = true)
 |-- edition_id: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- country_noc: string (nullable = true)
 |-- gold: integer (nullable = true)
 |-- silver: integer (nullable = true)
 |-- bronze: integer (nullable = true)
 |-- total: integer (nullable = true)



In [108]:
print('Top 3 ranking by own score:')
Olympic_Games_Medal_Tally \
    .withColumn('own score', (Olympic_Games_Medal_Tally.gold*3+Olympic_Games_Medal_Tally.silver*2+Olympic_Games_Medal_Tally.bronze*1)) \
    .select('edition', 'country', 'own score', 'gold', 'silver', 'bronze') \
    .sort('own score', ascending=False) \
    .show(3)

Top 3 ranking by own score:
+--------------------+-------------+----+------+------+
|             edition|      country|gold|silver|bronze|
+--------------------+-------------+----+------+------+
|1904 Summer Olympics|United States|  80|    85|    83|
|1980 Summer Olympics| Soviet Union|  80|    69|    46|
|1984 Summer Olympics|United States|  83|    61|    30|
+--------------------+-------------+----+------+------+
only showing top 3 rows



### Events

In [24]:
# Read the CSV file into a DataFrame
events = spark.read.csv("data/Olympic_Athlete_Event_Results.csv", header=True, inferSchema=True)

In [25]:
events.show()

+--------------------+----------+-----------+---------+--------------------+---------+---------------+----------+----------+-----+-----------+
|             edition|edition_id|country_noc|    sport|               event|result_id|        athlete|athlete_id|       pos|medal|isTeamSport|
+--------------------+----------+-----------+---------+--------------------+---------+---------------+----------+----------+-----+-----------+
|1908 Summer Olympics|         5|        ANZ|Athletics|     100 metres, Men|    56265|Ernest Hutcheon|     64710|       DNS| null|      false|
|1908 Summer Olympics|         5|        ANZ|Athletics|     400 metres, Men|    56313|   Henry Murray|     64756|       DNS| null|      false|
|1908 Summer Olympics|         5|        ANZ|Athletics|     800 metres, Men|    56338|  Harvey Sutton|     64808| 3 h8 r1/2| null|      false|
|1908 Summer Olympics|         5|        ANZ|Athletics|     800 metres, Men|    56338|    Guy Haskins|    922519|       DNS| null|      false|

In [26]:
import pyspark.sql.functions as F

df = events.select('event').distinct()
df = df.withColumn('output', F.regexp_replace('event', r', (Men|Women|Open|Team|Mixed)', ''))

print('Number of unique olympic sports in all of history:')
df.select('output').distinct().count()
# print('*if you split running into 100m, 200m, 4x100m, etc. ')


Number of unique olympic sports in all of history:


742

In [27]:
events.filter(events.event.contains('Basketball, Men')).show()

+--------------------+----------+-----------+----------+---------------+---------+--------------------+----------+---+-----+-----------+
|             edition|edition_id|country_noc|     sport|          event|result_id|             athlete|athlete_id|pos|medal|isTeamSport|
+--------------------+----------+-----------+----------+---------------+---------+--------------------+----------+---+-----+-----------+
|1992 Summer Olympics|        23|        ANG|Basketball|Basketball, Men|    32728|Benjamim João Romano|      6990| 10| null|       true|
|1992 Summer Olympics|        23|        ANG|Basketball|Basketball, Men|    32728|   Herlander Coimbra|      5899| 10| null|       true|
|1992 Summer Olympics|        23|        ANG|Basketball|Basketball, Men|    32728|Jean-Jacques da C...|      5907| 10| null|       true|
|1992 Summer Olympics|        23|        ANG|Basketball|Basketball, Men|    32728|  Victor de Carvalho|      5965| 10| null|       true|
|1992 Summer Olympics|        23|        

In [28]:
df = events.filter(events.medal.isNotNull())

df.show()

+--------------------+----------+-----------+---------+--------------------+---------+-----------------+----------+---+------+-----------+
|             edition|edition_id|country_noc|    sport|               event|result_id|          athlete|athlete_id|pos| medal|isTeamSport|
+--------------------+----------+-----------+---------+--------------------+---------+-----------------+----------+---+------+-----------+
|1908 Summer Olympics|         5|        ANZ|Athletics|3,500 metres Race...|    56421|       Harry Kerr|     64719|  3|Bronze|      false|
|1908 Summer Olympics|         5|        ANZ|   Boxing|   Middleweight, Men|    21263|      Snowy Baker|     45153|  2|Silver|      false|
|1908 Summer Olympics|         5|        ANZ|    Rugby|          Rugby, Men|    31505|     John Barnett|     11237|  1|  Gold|       true|
|1908 Summer Olympics|         5|        ANZ|    Rugby|          Rugby, Men|    31505|  Phil Carmichael|     11239|  1|  Gold|       true|
|1908 Summer Olympics|     

In [93]:
events.filter(events.result_id == 56421).show()

+--------------------+----------+-----------+---------+--------------------+---------+-------------------+----------+----------+------+-----------+
|             edition|edition_id|country_noc|    sport|               event|result_id|            athlete|athlete_id|       pos| medal|isTeamSport|
+--------------------+----------+-----------+---------+--------------------+---------+-------------------+----------+----------+------+-----------+
|1908 Summer Olympics|         5|        ANZ|Athletics|3,500 metres Race...|    56421|         Harry Kerr|     64719|         3|Bronze|      false|
|1908 Summer Olympics|         5|        ANZ|Athletics|3,500 metres Race...|    56421|     Albert Rowland|     64788|         5|  null|      false|
|1908 Summer Olympics|         5|        DEN|Athletics|3,500 metres Race...|    56421|Charles Vestergaard|     67007|         6|  null|      false|
|1908 Summer Olympics|         5|        DEN|Athletics|3,500 metres Race...|    56421|         Arne Højme|     6

In [92]:
print(f'There were a total of {events.filter(events.medal.isNotNull()).count()} events')
print(f'{} events were competing for a medal')
# events.filter(events.medal.isNull()).count()

There were a total of 44687 events


In [39]:
print('Most Gold medals by athlete:')
df.filter(df.medal == 'Gold').groupBy('athlete').agg({'medal': 'count'}).orderBy('count(medal)', ascending=False).show(3)

Most Gold medals by athlete:
+---------------+------------+
|        athlete|count(medal)|
+---------------+------------+
| Michael Phelps|          23|
|     Carl Lewis|           9|
|Larisa Latynina|           9|
+---------------+------------+
only showing top 3 rows



In [62]:
print('Most medals (Gold, Silver, Bronze) by athlete:')

df = df.withColumns({\
    "gold": (F.col("medal") == "Gold").cast('integer'), \
    "silver": (F.col("medal") == "Silver").cast('integer'), \
    "bronze": (F.col("medal") == "Bronze").cast('integer')
    })

df \
    .groupBy('athlete') \
    .agg(
        F.count('medal').alias('medal_count'),
        F.sum('gold').alias('gold'),
        F.sum('silver').alias('silver'),
        F.sum('bronze').alias('bronze')
    ) \
    .orderBy('medal_count', ascending=False) \
    .show(3)

Most medals (Gold, Silver, Bronze) by athlete:
+---------------+-----------+----+------+------+
|        athlete|medal_count|gold|silver|bronze|
+---------------+-----------+----+------+------+
| Michael Phelps|         28|  23|     3|     2|
|Larisa Latynina|         18|   9|     5|     4|
|  Marit Bjørgen|         15|   8|     4|     3|
+---------------+-----------+----+------+------+
only showing top 3 rows



### Results

In [29]:
results = spark.read.csv("data/Olympic_Results.csv", header=True, inferSchema=True)

In [30]:
results.filter(results.sport == 'Basketball').show()

+---------+-----------------+--------------------+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+
|result_id|      event_title|             edition|edition_id|     sport|           sport_url|         result_date|     result_location| result_participants|       result_format|result_detail|  result_description|
+---------+-----------------+--------------------+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+
|   303046|Basketball, Women|2012 Summer Olympics|        54|Basketball|/editions/54/spor...|28 July – 11 Augu...|Basketball Arena,...|143 from 12 count...|Round-robin pools...|           na|The United States...|
|   900000|  Basketball, Men|1904 Summer Olympics|         3|Basketball|/editions/3/sport...|   11 – 12 July 1904|Francis Field, Wa...| 40 from 1 co

### Hosts

In [101]:
hosts = spark.read.csv("data/Olympics_Games.csv", header=True, inferSchema=True)
countries = spark.read.csv("data/Olympics_Country.csv", header=True, inferSchema=True)

hosts = hosts.join(countries, hosts.country_noc == countries.noc, how='inner')


In [83]:
hosts.show()

+--------------------+----------+------------+----+----------------+--------------------+-----------+-----------+------------+--------------------+-------------------+---+-------------+
|             edition|edition_id| edition_url|year|            city|    country_flag_url|country_noc| start_date|    end_date|    competition_date|             isHeld|noc|      country|
+--------------------+----------+------------+----+----------------+--------------------+-----------+-----------+------------+--------------------+-------------------+---+-------------+
|1896 Summer Olympics|         1| /editions/1|1896|          Athina|https://olympedia...|        GRE|    6 April|    15 April|        6 – 13 April|               null|GRE|       Greece|
|1900 Summer Olympics|         2| /editions/2|1900|           Paris|https://olympedia...|        FRA|       null|        null| 14 May – 28 October|               null|FRA|       France|
|1904 Summer Olympics|         3| /editions/3|1904|       St. Louis|ht

In [103]:
print('Cities hosting the Olympics:')

df = hosts \
    .groupBy(hosts.city, hosts.country) \
    .agg(F.count('year').alias('number of times hosted'), F.collect_set('year').alias('years when hosted')) \
    .orderBy('number of times hosted', ascending=False)
    
df.limit(5).toPandas()

Cities hosting the Olympics:


,city,country,number of times hosted,years when hosted
0,London,Great Britain,4,"[1944, 1908, 1948, 2012]"
1,Athina,Greece,3,"[2004, 1896, 1906]"
2,Paris,France,3,"[1900, 2024, 1924]"
3,Los Angeles,United States,3,"[1932, 1984, 2028]"
4,Sankt Moritz,Switzerland,2,"[1948, 1928]"
